In [1]:
SAMPLE_INPUT = """
Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"""

In [2]:
from typing import Set, List, Dict
from pydantic import BaseModel

In [3]:
class Card(BaseModel):
	card_number: int
	winning_numbers: Set[int]
	possessed_numbers: Set[int]

	@classmethod
	def read(cls, text: str):
		data = {}
		card_number, numbers = text.split(": ")
		data['card_number'] = int(card_number.replace('Card ', ''))
		winning, possessed = numbers.split(" | ")
		data['winning_numbers'] = set(map(int, filter(bool, winning.split())))
		data['possessed_numbers'] = set(map(int, filter(bool, possessed.split())))
		return cls(**data)
	
	@property
	def match_count(self):
		return len(self.winning_numbers.intersection(self.possessed_numbers))
	
	@property
	def score(self):
		if self.match_count == 0:
			return 0
		return int(2 ** (self.match_count-1))

In [34]:
class CardList(BaseModel):
	cards: Dict[int, Card]

	@property
	def sorted_cards(self):
		return sorted(self.cards.values(), key=lambda card: card.card_number)

	@classmethod
	def read(cls, text: str):
		cards = {}
		for line in text.splitlines():
			if line:
				card = Card.read(line)
				cards[card.card_number] = card
		return cls(cards=cards)
	
	@property
	def score(self):
		return sum([card.score for card in self.sorted_cards])

In [26]:
CardList.read(SAMPLE_INPUT).score

13

In [27]:
input_text = open('input.txt').read()

In [28]:
CardList.read(input_text).score

23673

In [46]:
class NewCardList(CardList):
	copy_counts: Dict[int, int] = {}

	def model_post_init(self, __context):
		for card in self.cards.values():
			self.copy_counts[card.card_number] = 1

	def __add_to_card(self, card: Card, count: int):
		for j in range(count):
			copy_number = card.card_number+j+1
			if copy_number in self.cards:
				self.copy_counts[copy_number] += self.copy_counts[card.card_number]

	@property
	def score(self):
		for i, card in enumerate(self.sorted_cards):
			match_count = card.match_count
			self.__add_to_card(card, match_count)
		return sum(self.copy_counts.values())

In [47]:
NewCardList.read(SAMPLE_INPUT).score

30

In [48]:
NewCardList.read(input_text).score

12263631